# WB RecSys Project

# Общее описание проекта

Необходимо на основании взаимодействий пользователей с товарами предсказать следующие взаимодействия пользователей с товарами.

# Stage 3

- Сформировать обучающую выборку
- Спроектировать схему валидации с учетом специфики задачи
- Обосновать выбор способа валидации


# Preprocessing text_data

# Импорт библиотек

In [1]:
import numpy as np
import numpy.typing as npt

import polars as pl

import pandas as pd
import pyarrow.parquet as pq

import dill
import re 

import tqdm

from sklearn.decomposition import PCA

import torch
from transformers import AutoTokenizer, AutoModel


import warnings

warnings.filterwarnings("ignore")

# Сделаем автоподгрузку всех изменений при перепрогонке ячейки
%load_ext autoreload
%autoreload 2

%matplotlib inline

# Данные

Путь до данных

In [2]:
data_path = "../../data/closed/"
data_load_path = "../../data/load/"


## text data

### Чтение данных

In [ ]:
df_text = pl.scan_parquet(data_load_path + "text_data_69020_final.parquet")
df_text.schema

## Обработка цвета товара

In [ ]:
popular_colors = np.array(
    [
        "бежевый",
        "белый",
        "голубой",
        "желтый",
        "зеленый",
        "коричневый",
        "красный",
        "оранжевый",
        "розовый",
        "серый",
        "синий",
        "фиолетовый",
        "черный",
    ]
)
popular_colors

In [ ]:
colors_dict = {color: i + 1 for i, color in enumerate(popular_colors)}
colors_dict["другой"] = len(popular_colors) + 1

colors_dict[""] = len(popular_colors) + 10
colors_dict["[]"] = len(popular_colors) + 10
colors_dict["['']"] = len(popular_colors) + 10
colors_dict['[""]'] = len(popular_colors) + 10
colors_dict

In [6]:
def which_color(x: str):
    # using try\except
    # for case if x is not iterable
    # or has other type
    try:
        if isinstance(x, str):
            for c in popular_colors:
                if c in x:
                    return colors_dict[c]
            return colors_dict["другой"]
        else:
            for c in popular_colors:
                for xx in x:
                    if c in xx:
                        return colors_dict[c]
            return colors_dict["другой"]
    except:
        return colors_dict["другой"]

Save items_colors

In [7]:
# Save items_colors
df_text.select(["nm_id", "colornames"]).rename(
    {   
        "nm_id": "item_id",
        "colornames": "color",
    }
).with_columns(
    pl.col("color").map_elements(which_color)
).collect().write_parquet(data_path + "items_colors.parquet")

## Обработка characteristics

Характеристики представлены в виде словаря, в котором нас интересуют поля `'charcName'` и `'charcValues'`. 

Нужно получить пары
['charcName': 'charcValues']

In [4]:
# Словарь для парсинга колонки characteristics
chars_dict = {
    "длина юбки/платья": "length",
    "длина юбки\\платья": "length",
    "модель платья": "model",
    "назначение": "purpose",
    "покрой": "cut",
    "рисунок": "pattern",
    "тип карманов": "pocket",
    "тип ростовки": "height",
    "тип рукава": "sleeve",
    "состав": "material",
    "вид застежки": "closure",
    "вид застёжки": "closure",
    "вырез горловины": "neckline",
    "страна производства": "country",
}

In [5]:
# ---------------------------------------------------
# Словари для проверки подстрок и составления таблицы
# ---------------------------------------------------


# Застежка
closure_dict = {
    "без застежки": ["без", "нет"],
    "молния": ["молни"],
    "пуговица": ["пуговиц"],
    "завязка": ["завязк"],
    "пояс": ["пояс"],
    "шнуровка": ["шнур"],
    "резинка": ["резин"],
}

# Рисунок
pattern_dict = {
    "абстракция": ["абстрак"],
    "без рисунка": ["без", "однотон", "нет"],
    "горох": ["горох", "горош"],
    "клетка": ["клет"],
    "леопардовый": ["леопард"],
    "полоска": ["полос"],
    "фигуры": ["фигур", "геометр"],
    "цветы": ["цвет", "растен"],
}

# Назначение
purpose_dict = {
    "большие размеры": ["больш"],
    "вечернее": ["вечер"],
    "выпускной": ["выпуск"],
    "беремен": ["для беременн", "будущие мамы", "род"],
    "кормления": ["корм"],
    "крещения": ["крещ"],
    "домашнее": ["дом"],
    "повседневная": ["повседнев"],
    "свадьба": ["свадьб"],
    "пляж": ["пляж"],
    "новый год": ["новый"],
    "школа": ["школ"],
    "спорт": ["спорт"],
    "офис": ["офис"],
}

# Карманы
pocket_dict = {
    "в_шве": ["в шв", "бок"],
    "без_карманов": ["без", "нет"],
    "прорезные": ["прорез"],
    "потайные": ["тайн"],
    "накладные": ["наклад"],
}

# Рукава
sleeve_dict = {
    "без рукавов": ["без", "нет"],
    "длинные": ["дли"],
    "короткие": ["кор"],
    "3/4": ["3/4", "3\\4", "34", "3", "4"],
    "7/8": ["7/8", "7\\8", "78", "7", "8"],
}

# Длина
length_dict = {
    "миди": [
        "миди",
        "серед",
        "10",
        "11",
        "ниже",
        "по",
    ],
    "макси": [
        "макси",
        "длин",
        "12",
        "13",
        "14",
        "15",
        "16",
        "в пол",
        "пол",
    ],
    "мини": [
        "мини",
        "кор",
        "9",
        "8",
        "до",
        "выше",
        "корот",
    ],
}


# Модель
model_dict = {
    "футляр": ["футл"],
    "рубашка": ["рубаш"],
    "открытое": ["откр"],
    "запах": ["запах"],
    "прямое": ["прям"],
    "кожаное": ["кожа"],
    "свадьба": ["свад"],
    "лапша": ["лап"],
    "вязаное": ["вяз"],
    "комбинация": ["комб"],
    "футболка": ["футб"],
    "водолазка": ["водо"],
    "бохо": ["бохо"],
    "сарафан": ["сараф"],
    "пиджак": ["пидж"],
    "трапеция": ["трапец"],
    "мини": ["мини"],
    "макси": ["макси"],
    "миди": ["миди"],
    "свободное": ["свобод"],
    "а-силуэт": [
        "а-силуэт",
        "а- силуэт",
        "а -силуэт",
        "а силуэт",
        "асилуэт",
        "a-силуэт",
        "a- силуэт",
        "a -силуэт",
        "a силуэт",
        "aсилуэт",
    ],
    "туника": ["тун"],
    "приталеное": ["тален"],
    "поло": ["поло"],
    "парео": ["парео"],
}


# Покрой
cut_dict = {
    "асимметричный": ["асимметр"],
    "приталенный": ["притален"],
    "рубашечный": ["рубаш"],
    "свободный": ["свободн"],
    "укороченный": ["укороч"],
    "удлиненный": ["удлин"],
    "облегающий": ["облега"],
    "полуприлегающий": ["полуприлег"],
    "прямой": ["прям"],
    "а-силуэт": [
        "а-силуэт",
        "а силуэт",
        "а- силуэт",
        "асилуэт",
        "a-силуэт",
        "a силуэт",
        "a- силуэт",
        "aсилуэт",
    ],
    "оверсайз": ["овер", "over"],
    "трапеция": ["трапец"],
    "длинный": ["длин"],
}

# Тип ростовки
height_dict = {
    "для высоких": [
        "1,7",
        "1,8",
        "1,9",
        "2,0",
        "17",
        "18",
        "19",
        "20",
        "для выс",
        "длявыс",
    ],
    "для невысоких": [
        "1,2",
        "1,3",
        "1,4",
        "1,5",
        "12",
        "13",
        "14",
        "15",
        "невыс",
        "не выс",
        "для невыс",
        "для не выс",
        "дляневыс",
        "дляне выс",
    ],
    "для среднего роста": [
        "для средн",
        "длясредн",
        "1,6",
        "16",
        "средн",
    ],
    "для всех": [
        "для всех",
        "длявсех",
        "безогр",
        "без огр",
    ],
}

# Материал
material_dict = {
    "акрил": ["акрил"],
    "бамбук": ["бамбук"],
    "вискоза": ["вискоза"],
    "кашемир": ["кашемир"],
    "кожа": ["кожа"],
    "лайкра": ["лайкра"],
    "лен": ["лен"],
    "нейлон": ["нейлон"],
    "полиамид": ["полиамид"],
    "полиэстер": ["полиэстер"],
    "спандекс": ["спандекс"],
    "трикотаж": ["трикотаж"],
    "шерсть": ["шерсть"],
    "шелк": ["шелк"],
    "штапель": ["штапель"],
    "шифон": ["шифон"],
    "хлопок": ["хлопок"],
    "эластан": ["эластан"],
}

# Вырез
neckline_dict = {
    "круглый": ["круг"],
    "классический": ["классич"],
    "стойка": ["стойк"],
    "овал": ["овал"],
    "бретели": ["бретел"],
    "v-образный": ["v"],
    "сердечко": ["сердеч"],
    "американка": ["америк"],
    "фигурный": ["фигур"],
    "u-образный": ["u"],
    "гольф": ["гольф"],
    "хомут": ["хомут"],
    "лодочка": ["лодоч"],
    "отложной": ["отлож"],
    "кармен": ["кармен"],
    "бант": ["бант"],
    "капюшон": ["капюш"],
    "квадратный": ["квадра"],
    "открытый": ["откр", "спина", "плеч"],
    "с горлом": ["с горлом", "горло"],
}

# Страна
country_dict = {
    "Россия": ["россия", "russia"],
    "Беларусь": ["беларусь"],
    "Турция": ["турция"],
    "Франция": ["франция"],
    "Киргизия": ["киргизия"],
    "Китай": ["китай", "china"],
    "Италия": ["италия"],
    "Индия": ["индия"],
    "Бангладеш": ["бангладеш"],
    "Узбекистан": ["узбекистан"],
    "Вьетнам": ["вьетнам"],
    "Гонконг": ["гонконг"],
}

In [6]:
chars_formating_dicts = {
    "length": length_dict,
    "model": model_dict,
    "purpose": purpose_dict,
    "cut": cut_dict,
    "pattern": pattern_dict,
    "pocket": pocket_dict,
    "height": height_dict,
    "sleeve": sleeve_dict,
    "material": material_dict,
    "closure": closure_dict,
    "neckline": neckline_dict,
    "country": country_dict,
}

In [7]:
def parse_char_dict(characteristics: dict):
    filtered_chars = {}
    for char in characteristics:
        try:
            filtered_chars[chars_dict[str.lower(char["charcName"])]] = [
                str.lower(x) for x in char["charcValues"]
            ]
        except:
            pass
    return filtered_chars


def get_char_value(characteristics: dict, char: str):
    try:
        return characteristics[char]
    except:
        return ["unknown"]

def format_chars(chars: npt.ArrayLike, char_dict: dict):

    for charType, charValues in char_dict.items():
        if any(v in "".join(chars) for v in charValues):
            return charType

    return "unknown"


In [ ]:
df_chars = pl.DataFrame()

for id_batch in pq.read_table(data_load_path + "text_data_69020_final.parquet").to_batches():
    df = id_batch.to_pandas()[["nm_id", "characteristics"]]

    df = df.rename(columns={"nm_id": "item_id"})

    df["characteristics"] = df["characteristics"].apply(parse_char_dict)

    for char in sorted(list(set(chars_dict.values()))):
        df[char] = df["characteristics"].apply(lambda x: get_char_value(x, char))

    for k, v in chars_formating_dicts.items():
        df[k] = df[k].apply(lambda x: format_chars(x, v))

    df = df.drop(columns="characteristics")

    df_chars = pl.concat([df_chars, pl.from_pandas(df)])

df_chars

In [11]:
df_chars.write_parquet(data_path + "df_chars.parquet")

## Текст

In [8]:
# df_descrs = pl.DataFrame()

# uncomment lines if data is too large and needs batches
# to fit memory

# item_batches = np.array_split(df_text.select("nm_id").collect().to_numpy().flatten(), 4)

# for id_batch in item_batches:

    # tmp = 
(
    df_text.select(["nm_id", "title", "description"])
    # .filter(pl.col("nm_id").is_in(id_batch))
    .rename({"nm_id": "item_id"})
    .with_columns(
        # Formatting titles
        pl.when(pl.col("title").is_not_null())
        .then(pl.col("title").str.replace(r"\s\s+", " ").str.to_lowercase())
        .otherwise(pl.col("title").map_elements(lambda x: "")),
        # Formatting descriptions
        pl.when(pl.col("description").is_not_null())
        .then(pl.col("description").str.replace(r"\s\s+", " ").str.to_lowercase())
        .otherwise(pl.col("description").map_elements(lambda x: "")),
        # Get len of titles in chars
        pl.col("title").str.len_chars().alias("title_len"),
        # Get len of descriptions in chars
        pl.col("description").str.len_chars().alias("descr_len"),
        # Get len of titles in words
        pl.col("title").str.split(by=" ").list.len().alias("title_word_len"),
        # Get len of descriptions in words
        pl.col("description").str.split(by=" ").list.len().alias("descr_word_len"),
    )
    .collect()
# save as parquet
).write_parquet(data_path + "df_descrs.parquet")

    # df_descrs = pl.concat([df_descrs, tmp])

#### Получим embeddings для описания

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Возьмем предобченную модель
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny").to(device)

In [ ]:
# разобъем на батчи для подачи в модель
descrs = np.array_split(
    pl.scan_parquet(data_path + "df_descrs.parquet")
    .select("description")
    .collect()
    .to_numpy()
    .flatten(),
    60 * 4,
)
len(descrs), len(descrs[5])

In [ ]:
all_embeddings = []

for i in tqdm.tqdm(range(len(descrs))):

    sentences = descrs[i].tolist()

    encoded_input = tokenizer(
        sentences,
        padding=True,
        truncation=True,
        max_length=124,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        model_output = model(**encoded_input)

    embeddings = model_output.pooler_output
    embeddings = torch.nn.functional.normalize(embeddings).to("cpu")

    all_embeddings.append(embeddings)

In [ ]:
all_embeddings

In [ ]:
# Сохраним в бинарник
with open(data_path + "descrs_embs.dill", "wb") as f:
    dill.dump(torch.cat(all_embeddings).numpy(), f)

### Снизим размерность

In [9]:
# Загрузим данные
with open(data_path + "descrs_embs.dill", "rb") as f:
    all_embeddings = dill.load(f)

In [10]:
components_to_keep = 10

In [ ]:
pca_lowrank = PCA(n_components=components_to_keep)
all_embeddings = pca_lowrank.fit_transform(all_embeddings)
all_embeddings.shape

In [ ]:
txt_embs_df = pl.DataFrame(
    all_embeddings,
    schema=[f"txt_emb_pca_{i}" for i in range(components_to_keep)],
)
txt_embs_df

In [13]:
# Concat with main table and save to parquet
pl.concat(
    [
        pl.scan_parquet(data_path + "df_descrs.parquet").collect(),
        txt_embs_df,
    ],
    how="horizontal",
).write_parquet(data_path + "df_descrs.parquet")

Названия товаров решено отбросить, т.к. по сути все товары являются платьями, 
и лишь в названиях небольшого количества товаров, содержится иформация дублируемая в описании 

## Brands

In [14]:
(
    df_text.select(["nm_id", "brandname"])
    .rename(
        {
            "nm_id": "item_id",
            "brandname": "brand",
        }
    )
    .with_columns(pl.col("brand").str.to_lowercase())
    .collect()
).write_parquet(data_path + "df_brands.parquet")

## Смерджим данные по айтемам

In [22]:
pl.scan_parquet(data_path + "df_brands.parquet").join(
    other=pl.scan_parquet(data_path + "df_chars.parquet"),
    on="item_id",
).join(
    other=pl.scan_parquet(data_path + "items_colors.parquet"),
    on="item_id",
).join(
    other=pl.scan_parquet(data_path + "df_descrs.parquet").drop(
        ["title", "description"]
    ),
    on="item_id",
).collect().write_parquet(
    data_path + "df_items.parquet"
)

In [ ]:
pl.scan_parquet(data_path + "df_items.parquet").collect()

Размерность та же, следовательно, ничего не потеряли)